In [1]:
from datetime import datetime, timezone
import pandas as pd
import numpy as np
from selenium import webdriver
from airtable import Airtable
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

In [2]:
def generate_delay(mean = 0.6, sigma = 0.1, mini = 0.5):
    """
    retourn un chiffre aléatoire suivant une distribution normal de moyenne 4 secondes et d'ecart-type de 0.8
    Ce chiffre aléatoire est le délai après chaque clique. Le but est de simuler le comportement d'un humain :
    un délai fixe peut attirer l'attention des contrôleurs, tout comme un délai aléatoire d'une distribution uniforme
    """
    mean = 0.6
    sigma = 0.1
    delai = abs(np.random.normal(mean,sigma,1)[0])
    while delai<mini:
        delai = abs(np.random.normal(mean,sigma,1)[0])
    time.sleep(delai)
    
    
def click_and_delai_xpath(xpath,mean = 0.6, sigma = 0.1, mini = 0.5):
    driver.find_element_by_xpath(xpath).click()
    generate_delay(mean, sigma, mini)

def from_cn(class_name):
    ele_sel =  driver.find_element_by_class_name(class_name)
    if ele_sel != None:
        return ele_sel.text
    else:
        return None

def is_ld_profile(url):
    if r".com/in/" in url:
        return True
    else:
        return False
    
def write_message(message):
    text_area = driver.find_element_by_xpath('//textarea[@name="message"]')
    for letter in message:
        time.sleep(abs(random.random()/150))  # sleep between 1 and 3 seconds
        text_area.send_keys(letter)

## Login

In [3]:
airtable = Airtable("appsukCYzNmwFtrh9", "linkedin", api_key='key1qvdsKWuaiJxKk')
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/")
generate_delay()

In [4]:
driver.maximize_window()

# #s'identtfier
click_and_delai_xpath('//a[@class="nav__button-secondary"]')
generate_delay(0.2, 0.005, 0.05)
driver.find_element_by_xpath('//input[@id="username"]').send_keys('emmanuel.thiery.pro@gmail.com')
generate_delay(0.2, 0.005, 0.05)
driver.find_element_by_xpath('//input[@id="password"]').send_keys('waitareum91')
generate_delay(0.2, 0.005, 0.05)


click_and_delai_xpath('//button[@type="submit"]')


    
    
### getting data

In [5]:
# df = pd.DataFrame()

In [6]:
# def get_position_company():
#     try:
#         job_position = driver.find_element_by_xpath('//a[@data-control-name="background_details_company"]//h3').text
#     except:
#         job_position = from_cn('t-18', )
        
#     try:
#         company = driver.find_elements_by_xpath('//a[@data-control-name="background_details_company"]//p')[1].text
#     except:
#         company = ""
        
#     return job_position, company
    
# def get_data(df):
#     action = True
#     if action:
#         d = {}
#         d["name"] = from_cn('t-24')
#         d["linkedin_link"] = driver.current_url
#         d["country"] = driver.find_element_by_xpath('//li[@class="t-16 t-black t-normal inline-block"]').text.split()[-1]
#         d['job_position'] = driver.find_element_by_xpath('//div[@class="flex-1 mr5"]/div/h2').text
#         #issue with job position https://www.linkedin.com/in/jennifer-lin-0a7432171/
#         d['company'] = driver.find_element_by_xpath('//ul[@class="pv-top-card--experience-list"]/li').text
#         d['contacted_in'] = datetime.now(timezone.utc).astimezone().isoformat()
              
#               #             "country":driver.find_elements_by_xpath('//a[@data-control-name="background_details_company"]//h4')[2].,
            
# #             "job_position":driver.find_element_by_xpath('//a[@data-control-name="background_details_company"]//h3').text,
# #             "contacted":"",
# #             "linkedin_link":,
#         print(d)
#         df = df.append(d, ignore_index = True)
#         return d
    
    
def get_data(df):
    action = True
    if action:
        d = {}
        d["name"] = from_cn('t-24')
        d["linkedin_link"] = driver.current_url
        d["country"] = driver.find_element_by_xpath('//li[@class="t-16 t-black t-normal inline-block"]').text.split()[-1]
#         d['job_position'] = driver.find_element_by_xpath('//div[@class="flex-1 mr5"]/div/h2').text
        d['job_position'] = driver.find_element_by_xpath('//a[@data-control-name="background_details_company"]/div/h3').text
        
        
        d['company'] = driver.find_element_by_xpath('//ul[@class="pv-top-card--experience-list"]/li').text
        d['contacted_in'] = datetime.now(timezone.utc).astimezone().isoformat()
              
              #             "country":driver.find_elements_by_xpath('//a[@data-control-name="background_details_company"]//h4')[2].,
            
#             "job_position":driver.find_element_by_xpath('//a[@data-control-name="background_details_company"]//h3').text,
#             "contacted":"",
#             "linkedin_link":,
        print(d)
#         df = df.append(d, ignore_index = True)
        return d

#### function for entreprise

In [7]:
def make_a_search(text):
    ele = driver.find_element_by_class_name('search-global-typeahead__input')
    ele.clear()
    ele.send_keys(text)
    
def get_connectable_and_picture_profile_list(whole_list):
    def connectable_and_picture(profile):
        try:
            profile.find_element_by_tag_name("footer")
            image_s = profile.find_element_by_tag_name("img")
            if not image_s.get_attribute('src').startswith("data:image"):
                return True
            else:
                return False
        except:
            return False
    
    connectable_list = [profile for profile in whole_list if connectable_and_picture(profile)]

    return connectable_list

def get_specific_profile(whole_list, text_lower_case, text_lower_case2):
    return [profile for profile in whole_list if (text_lower_case or text_lower_case2)  in profile.text.lower()]

#### function for message


In [8]:
def tag_click(tag, texte, click = True):
    try:
        if click:
            driver.find_element_by_xpath(f'//{tag}[text()="{texte}"]').click()
    except:
        return driver.find_element_by_xpath(f'//{tag}[text()="{texte}"]').text

    
def can_connect(d):
    try:
        driver.find_element_by_xpath('//span[text()="En attente"]').text
        
        print("{} already request sent".format(d["name"]))
        return False
    except:
        driver.find_element_by_xpath('//span[text()="Se connecter"]').click()
        return True
    
def sent_message(message, d):
    #click on "plus"
    driver.find_element_by_xpath('/html/body/div[8]/div[3]/div/div/div/div/div[2]/main/div[1]/section/div[2]/div[1]/div[2]/div/div/div/div/button').click()
    generate_delay()
    con_possible = can_connect(d)
    print(con_possible)
    generate_delay()
    if con_possible:
        tag_click("span", "Ajouter une note")
        write_message(message)
        airtable.insert(d)
        generate_delay()

def generate_message(d):
    full_name = d["name"]
    if "Dr." in full_name:
        name = "Dr. " + full_name.split()[-1] # take the last name of the list, likely the nom
    else:
        name = full_name.split()[0] # take the prenom
    message_en = f"Bonjour {name},\nI hope the covid crisis doesn't affect you too much. I was wondering if you already used electrosynthesis to produce one of your APIs ? "
    message_fr = f"Bonjour {name},\nNous sommes tous les deux experts dans l'industrie pharmaceutique et je pense qu'il serait intéressant d'échanger à ce sujet, qu'en dites-vous ?"

    if d["country"] == "France":
        return message_fr
    else:
        return message_en

def scrap_profile_and_send_message(df):
    d = get_data(df)
    message = generate_message(d)
    sent_message(message, d)
    

    


## recheche entreprise 

In [9]:
driver.switch_to.window(driver.window_handles[0])
    
make_a_search("Sensient")

In [10]:
# time.sleep(1)
ele = driver.find_element_by_class_name('search-global-typeahead__input')
ele.send_keys(Keys.DOWN)

In [11]:
ele = driver.find_element_by_class_name('search-global-typeahead__input')
ele.send_keys(Keys.ENTER)

## on profil entreprise, display employee profiles

In [10]:
tag_click("a", "Personnes")

### scroll to he bottom

In [33]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [34]:
profiles_list = driver.find_elements_by_class_name('artdeco-card')
len(profiles_list)

660

In [35]:
# %%time


connectable_profiles = get_connectable_and_picture_profile_list(profiles_list)
len(connectable_profiles)

459

In [36]:
short_list = get_specific_profile(connectable_profiles, "product", "r&d")
len(short_list)

17

In [38]:
for profile in short_list:
    print(profile.text)
#     driver.switch_to.window(driver.window_handles[0])
    driver.execute_script("arguments[0].scrollIntoView();", profile)
    generate_delay(0.2,0.05)
    driver.execute_script("window.scrollTo(0, window.scrollY - 150)")
    generate_delay(0.2,0.05)
    link = profile.find_element_by_tag_name('a').get_attribute('href')
    print(link)
    driver.execute_script(f'''window.open("{link}","_blank");''')
    generate_delay(0.7,0.15)
driver.switch_to.window(driver.window_handles[0])

sylvain grimault
relation de 3e niveau
· 3e
assistant responsable production at Sensient Technologies Corporation
Se connecter
https://www.linkedin.com/in/sylvain-grimault-951461119/
Joel Esteban Casado
relation de 3e niveau
· 3e
Product Manager Essential Oils en Sensient Technologies Corporation
Se connecter
https://www.linkedin.com/in/joelesteban/
Lisha Daniel
relation de 3e niveau
· 3e
R&D Management / Fermentation / Innovation / New Product Development
Se connecter
https://www.linkedin.com/in/lishajacob/
Emanuela Consuelo Epis
relation de 3e niveau
· 3e
Product Manager Animal Nutrition presso Prosol spa
Se connecter
https://www.linkedin.com/in/emanuela-consuelo-epis-b30a315b/
Ken Schmidt
relation de 3e niveau
· 3e
Warehouse/Logistics/Production Supervisor at Sensient Technologies
Se connecter
https://www.linkedin.com/in/ken-schmidt-01053743/
Nick Laird
relation de 3e niveau
· 3e
Site Production Manager at Sensient Flavors Ltd
Se connecter
https://www.linkedin.com/in/nick-laird-2ab5

### Sent message

In [39]:
for tab in driver.window_handles:
    driver.switch_to.window(tab)
    generate_delay(mean = 0.6, sigma = 0.1)
    print(driver.current_url)
    if is_ld_profile(driver.current_url):
        anwser = input('oui/non\n')
        if anwser == "oui":
            scrap_profile_and_send_message(df)
        else:
            driver.close()
        

https://www.linkedin.com/company/sensient/people/
https://www.linkedin.com/in/windylyn-a-634186a2/
oui/non
oui
{'name': 'Windylyn A.', 'linkedin_link': 'https://www.linkedin.com/in/windylyn-a-634186a2/', 'country': 'Philippines', 'job_position': 'Production Planner', 'company': 'Sensient Technologies Corporation', 'contacted_in': '2020-06-28T22:20:35.366246+02:00'}
True
https://www.linkedin.com/in/sheryl-encarnacion-a98019a2/
oui/non
oui
{'name': 'Sheryl Encarnacion', 'linkedin_link': 'https://www.linkedin.com/in/sheryl-encarnacion-a98019a2/', 'country': 'Philippines', 'job_position': 'Product Development Specialist', 'company': 'San Miguel Corporation', 'contacted_in': '2020-06-28T22:20:56.970732+02:00'}
True
https://www.linkedin.com/in/andrew-grace-79648512/
oui/non
oui
{'name': 'Andrew Grace', 'linkedin_link': 'https://www.linkedin.com/in/andrew-grace-79648512/', 'country': 'Australie', 'job_position': 'Product Manager', 'company': 'Sensient Technologies Corporation', 'contacted_in'

In [100]:
# driver.switch_to.window(driver.window_handles[0])
get_data(df)

{'name': 'Giancarlo Porta', 'linkedin_link': 'https://www.linkedin.com/in/giancarlo-porta-28a7b828/', 'country': 'Suisse', 'job_position': 'Export Manager (Eastern Europe, CIS countries)', 'company': 'Refarmed Chemicals', 'contacted_in': '2020-06-18T15:38:37.960519+02:00'}


{'name': 'Giancarlo Porta',
 'linkedin_link': 'https://www.linkedin.com/in/giancarlo-porta-28a7b828/',
 'country': 'Suisse',
 'job_position': 'Export Manager (Eastern Europe, CIS countries)',
 'company': 'Refarmed Chemicals',
 'contacted_in': '2020-06-18T15:38:37.960519+02:00'}

In [54]:
driver.find_element_by_xpath('//li[@class="t-16 t-black t-normal inline-block"]').text.split()[-1]

'Allemagne'

In [49]:


# def get_position_company():
#     try:
#         job_position = driver.find_element_by_xpath('//a[@data-control-name="background_details_company"]//h3').text
#     except:
#         job_position = from_cn('t-18', )
        
#     try:
#         company = driver.find_elements_by_xpath('//a[@data-control-name="background_details_company"]//p')[1].text
#     except:
#         company = ""
        
#     return job_position, company
    



In [59]:
driver.switch_to.window(driver.window_handles[-5])
# get_data(df)
get_data(df)

{'name': '劉育綾 (Catherine Liu)', 'linkedin_link': 'https://www.linkedin.com/in/%E8%82%B2%E7%B6%BE-%E5%8A%89-7b30432a/', 'country': 'Taiwan', 'job_position': 'QA Specialist', 'company': 'Sci Pharmtech Inc', 'contacted_in': '2020-06-18T14:52:32.357575+02:00'}


{'name': '劉育綾 (Catherine Liu)',
 'linkedin_link': 'https://www.linkedin.com/in/%E8%82%B2%E7%B6%BE-%E5%8A%89-7b30432a/',
 'country': 'Taiwan',
 'job_position': 'QA Specialist',
 'company': 'Sci Pharmtech Inc',
 'contacted_in': '2020-06-18T14:52:32.357575+02:00'}

In [60]:
driver.find_element_by_xpath('//div[@class="flex-1 mr5"]/div/h2').text

'Project Management Bioscience Solutions bei Kraeber GmbH & Co'

In [42]:
driver.quit()